### 2.Problem Statement
### It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

In [1]:
# Solution
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                   'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                   'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


#### 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [2]:
for i in range(1, df['FlightNumber'].count()+1):
      if pd.isnull(df.loc[i,'FlightNumber']):
            df.loc[i, 'FlightNumber'] = df.loc[i-1, 'FlightNumber'] + 10
df['FlightNumber'] = df['FlightNumber'].astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


#### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [3]:
df_temp=df.copy()
df_temp[['From','To']] = df_temp.From_To.str.split(pat='_',expand=True)
df_temp

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045,"[23, 47]",KLM(!),LoNDon,paris
1,MAdrid_miLAN,10055,[],<Air France> (12),MAdrid,miLAN
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,PaRis
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,londOn


#### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [4]:
df_temp.From_To=df_temp.From_To.str.capitalize()
df_temp.From=df_temp.From.str.capitalize()
df_temp.To=df_temp.To.str.capitalize()
df_temp

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,London_paris,10045,"[23, 47]",KLM(!),London,Paris
1,Madrid_milan,10055,[],<Air France> (12),Madrid,Milan
2,London_stockholm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_paris,10075,[13],12. Air France,Budapest,Paris
4,Brussels_london,10085,"[67, 32]","""Swiss Air""",Brussels,London


#### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [5]:
df.drop('From_To',axis=1,inplace=True)
df

,FlightNumber,RecentDelays,Airline
0,10045,"[23, 47]",KLM(!)
1,10055,[],<Air France> (12)
2,10065,"[24, 43, 87]",(British Airways. )
3,10075,[13],12. Air France
4,10085,"[67, 32]","""Swiss Air"""


In [6]:
df['From_To']=df_temp['From_To']
df

,FlightNumber,RecentDelays,Airline,From_To
0,10045,"[23, 47]",KLM(!),London_paris
1,10055,[],<Air France> (12),Madrid_milan
2,10065,"[24, 43, 87]",(British Airways. ),London_stockholm
3,10075,[13],12. Air France,Budapest_paris
4,10085,"[67, 32]","""Swiss Air""",Brussels_london


#### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.
#### Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [7]:
df
rows = []
_ = df.apply(lambda row:[rows.append([row['Airline'], row['FlightNumber'],nn,row['From_To']]) 
                         for nn in row.RecentDelays], axis=1)
rows

[['KLM(!)', 10045, 23, 'London_paris'],
 ['KLM(!)', 10045, 47, 'London_paris'],
 ['(British Airways. )', 10065, 24, 'London_stockholm'],
 ['(British Airways. )', 10065, 43, 'London_stockholm'],
 ['(British Airways. )', 10065, 87, 'London_stockholm'],
 ['12. Air France', 10075, 13, 'Budapest_paris'],
 ['"Swiss Air"', 10085, 67, 'Brussels_london'],
 ['"Swiss Air"', 10085, 32, 'Brussels_london']]

In [8]:
df_new=pd.DataFrame(rows, columns=['Airline','FlightNumber','RecentDelays','From_To'])
df_new

,Airline,FlightNumber,RecentDelays,From_To
0,KLM(!),10045,23,London_paris
1,KLM(!),10045,47,London_paris
2,(British Airways. ),10065,24,London_stockholm
3,(British Airways. ),10065,43,London_stockholm
4,(British Airways. ),10065,87,London_stockholm
5,12. Air France,10075,13,Budapest_paris
6,"""Swiss Air""",10085,67,Brussels_london
7,"""Swiss Air""",10085,32,Brussels_london


In [9]:
df1 = pd.DataFrame(df['RecentDelays'].values.tolist())
df1

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [10]:
col_dict={i:"Delay"+str(i+1) for i in range(df1.shape[1])}
col_dict

{0: 'Delay1', 1: 'Delay2', 2: 'Delay3'}

In [11]:
df1.rename(columns=col_dict,inplace=True)
df1

,Delay1,Delay2,Delay3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [12]:
df[["Delay1","Delay2","Delay3"]] = df1[["Delay1","Delay2","Delay3"]]
df.drop('RecentDelays',axis=1,inplace=True)
df

,FlightNumber,Airline,From_To,Delay1,Delay2,Delay3
0,10045,KLM(!),London_paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid_milan,NaN,NaN,NaN
2,10065,(British Airways. ),London_stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest_paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels_london,67.0,32.0,NaN
